# Tartu Ülikooli ÕIS2 andmed

See märkmik (notebook) pärib aineinfot Tartu Ülikooli ÕIS2 avalikust API-st.

**Eesmärk:** Luua terviklik andmestik (`course_details.csv`), mis sisaldab ainete kirjeldusi, õpiväljundeid ja eeldusaineid ja kõike muud, mida saaks kasutada edasises projektis.

**Protsess:**
1.  **Hangi ainete nimekiri:** Pärime API-st kõikide ainete ID-d (kasutades lehekülgede kaupa pärimist).
2.  **Defineeri abifunktsioonid:** Funktsioonid keerulise JSON-struktuuri lihtsustamiseks ja teksti vormindamiseks.
3.  **Hangi detailne info:** Käime tsükliga läbi kõik ained, pärime nende viimase versiooni (aine kava) ja salvestame andmed.

In [ ]:
# Impordime teegid
import requests
import json
import pandas as pd
import time
import traceback

# Seadistused
API_BASE_URL = "https://ois2.ut.ee/api/courses"
COURSE_LIST_FILE = "../andmed/ids.csv"
COURSE_DETAILS_FILE = "../andmed/toorandmed.csv"

## Samm 1: Ainete nimekirja hankimine

Meie esimene ülesanne on teada saada, **mis ained üldse ÕIS2 süsteemis eksisteerivad**.

Kuna Tartu Ülikoolis on tuhandeid aineid, ei luba server meil neid kõiki ühe hiiglasliku päringuga alla laadida – see koormaks süsteemi üle ja päring aeguks (*timeout*). Seetõttu peame kasutama lähenemist, mida nimetatakse **lehekülgedeks jaotamiseks (pagination)**.

**Kuidas see töötab?**
1.  **Tükeldamine (*Chunking*):** Me küsime serverilt andmeid väikeste portsude kaupa (siin `TAKE = 200`).
2.  **Tehniline eripära (GET vs POST):** ÕIS2 API on ehitatud nii, et **esimene leht** (algus) tuleb küsida `GET` meetodiga. Kõik **järgmised lehed** (kui liigume nimekirjas edasi) tuleb küsida `POST` meetodiga. Seetõttu näed koodis kahte erinevat funktsiooni andmete pärimiseks.
3.  **Tsükkel:** Me kordame pärimist seni, kuni server vastab tühja nimekirjaga või annab märku, et ained on otsas.

**Mis andmeid me saame?**
Selles sammus ei lae me veel alla aine pikki kirjeldusi ega õpiväljundeid (see teeks töö liiga aeglaseks).
* **Saame:** Aine "visiitkaardi": unikaalse ID (`course_uuid`), ainekoodi (nt `MTAT.03.227`) ja viite aineprogrammi viimasele versioonile (`latest_version_uuid`).
* **Ei saa veel:** Aine sisu, hindamiskriteeriume, õppejõude (need laeme Sammus 3).

**Kuhu info salvestatakse?**
* **Töö käigus (mälus):** Iga lehekülje andmed lisatakse jooksvalt Pythoni muutujasse `all_courses` (see on list).
* **Töö lõpus (failis):** Kui kõik ained on leitud, salvestame nimekirja faili **`course_ids.csv`**. See toimib meie jaoks "vahefinišina" – järgmises etapis loeme aineid juba sellest failist.

In [4]:
# --- SEADISTUSED ---
def get_initial_chunk():
    """
    Pärib esimese lehe aineid (GET meetodiga).
    ÕIS2 nõuab alguseks GET päringut.
    """
    r = requests.get(API_BASE_URL, timeout=30)
    r.raise_for_status()
    return r.json()

def post_next_chunk(start_offset, take=200):
    """
    Pärib järgmised lehed (POST meetodiga).
    Suuremate nihete (offset) puhul on vaja POST päringut.
    """
    payload = {
        "start": start_offset, 
        "take": take
    }
    r = requests.post(
        API_BASE_URL,
        headers={"Content-Type": "application/json", "Accept": "application/json"},
        data=json.dumps(payload),
        timeout=60
    )
    r.raise_for_status()
    return r.json()

# --- KÄIVITAMINE ---
all_courses = []
seen_uuids = set()
CHUNK_SIZE = 200

print("Alustan ainenimekirja laadimist...")

# 1. SAMM: Esimene päring (GET)
# See on vajalik, sest API keeldub POST päringust, kui nihe on 0.
try:
    chunk = get_initial_chunk()
    
    # Töötleme esimest portsu
    for c in chunk:
        if c["uuid"] not in seen_uuids:
            seen_uuids.add(c["uuid"])
            all_courses.append({
                "course_uuid": c["uuid"],
                "course_code": c.get("code"),
                "latest_version_uuid": c.get("latest_version_uuid"),
            })
            
    # Arvutame uue nihke (tavaliselt on esimese päringu suurus 20)
    offset = len(chunk)
    print(f"Esimene leht laetud (GET). Järgmine nihe: {offset}...")

except Exception as e:
    print(f"Kriitiline viga esimese päringuga: {e}")
    chunk = [] # Et tsükkel ei käivituks vigaselt

# 2. SAMM: Järgmised päringud (POST tsükkel)
while True:
    # Kui eelmine päring oli tühi, siis on kõik ained käes
    if not chunk:
        break

    try:
        chunk = post_next_chunk(offset, CHUNK_SIZE)
    except Exception as e:
        print(f"Viga andmeploki laadimisel nihkega {offset}: {e}")
        break

    if not chunk:
        print("Server tagastas tühja vastuse. Nimekiri on lõpus.")
        break

    for c in chunk:
        if c["uuid"] not in seen_uuids:
            seen_uuids.add(c["uuid"])
            all_courses.append({
                "course_uuid": c["uuid"],
                "course_code": c.get("code"),
                "latest_version_uuid": c.get("latest_version_uuid"),
            })

    actual_count = len(chunk)
    offset += actual_count
    print(f"Laetud veel {actual_count} ainet... (Kokku: {len(all_courses)})", end="\r")
    
    # Turvavõrk: kui server andis vähem kui küsisime, on tõenäoliselt lõpp,
    # aga laseme while-tsüklil joosta tühja vastuseni (if not chunk), et olla kindel.

print(f"\n\n--- TULEMUS ---")
print(f"Kokku leiti: {len(all_courses)} ainet.")
print(f"Unikaalseid ID-sid: {len(seen_uuids)}")

# Salvestame
df_ids = pd.DataFrame(all_courses)
df_ids.to_csv(COURSE_LIST_FILE, index=False)
print(f"Nimekiri salvestatud faili: {COURSE_LIST_FILE}")

Alustan ainenimekirja laadimist...
Esimene leht laetud (GET). Järgmine nihe: 20...
Server tagastas tühja vastuse. Nimekiri on lõpus.


--- TULEMUS ---
Kokku leiti: 3031 ainet.
Unikaalseid ID-sid: 3031
Nimekiri salvestatud faili: course_ids_TEST.csv


## Samm 2: Tööriistade ettevalmistamine (abifunktsioonid)

Enne kui saame 3. sammus hakata tuhandeid aineid alla laadima, peame ehitama endale **"tööriistakasti"**. Toores andmekuju, mis serverist tuleb, ei sobi kahjuks otse meie lõppeesmärgi (CSV tabeli) jaoks.

Selles sammus defineerime Pythoni funktsioonid, mis lahendavad kolm peamist probleemi:

1.  **Probleem: "Pesastatud" andmed (*Nested JSON*)**
    * **Olukord:** Server saadab info hierarhilise puuna (sõnastik sõnastiku sees). Näiteks on aine pealkiri peidetud sügavale: `aine -> üldinfo -> pealkirjad -> inglise_k`.
    * **Lahendus:** Funktsioon `flatten_json` teeb selle puu "lamedaks". See tõstab kõik andmed ühele tasandile, et saaksime neid hiljem mugavalt Exceli või Pandas tabeli veergudena kasutada (nt tekib veerg `general__title__en`).

2.  **Probleem: Aine ajalugu ja versioonid**
    * **Olukord:** Ühel ainel (näiteks "Programmeerimine") on ajas palju erinevaid aineprogramme (versioone). Meid ei huvita 2015. aasta iganenud sisu.
    * **Lahendus:** Funktsioon `pick_latest_version` vaatab läbi kõik aine versioonid ja valib välja kõige uuema/kinnitatud versiooni, et saaksime kõige värskema info.

3.  **Probleem: Teksti loetavus**
    * **Olukord:** Õpiväljundid tulevad tihti keerulise koodi-teksti seguna.
    * **Lahendus:** Funktsioon `bulletify` võtab need nimekirjad ja teeb neist ilusa ja loetava teksti (bullet-punktidega), mis sobib hästi lugemiseks või keelemudelitele sisendiks.

**NB!** Selle lahtri käivitamine ei lae veel midagi alla ega väljasta midagi ekraanile. See lihtsalt salvestab need funktsioonid mällu, et oleksime järgmiseks sammuks valmis.

In [5]:
# --- API päringud ---

def fetch_course_details(uuid: str) -> dict:
    """Pärib aine üldise konteineri andmed."""
    r = requests.get(f"{API_BASE_URL}/{uuid}", timeout=30)
    r.raise_for_status()
    return r.json()

def fetch_versions(course_uuid: str) -> list[dict]:
    """Pärib kõik aine versioonid (aineprogrammid) antud ainele."""
    r = requests.get(f"{API_BASE_URL}/{course_uuid}/versions", timeout=30)
    r.raise_for_status()
    return r.json()

# --- Andmetöötluse abifunktsioonid ---

def is_scalar(x):
    """Kontrollib, kas väärtus on lihtne number, tekst või tõeväärtus."""
    return isinstance(x, (str, int, float, bool)) or x is None

def flatten_json(obj, parent_key="", sep="__"):
    """
    Teeb rekursiivselt pesastatud sõnastikud lamedaks.
    Näide: {'overview': {'en': 'text'}} muutub kujule {'overview__en': 'text'}
    """
    flat = {}
    if isinstance(obj, dict):
        for k, v in obj.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            flat.update(flatten_json(v, new_key, sep))
    elif isinstance(obj, list):
        # Kui listis on ainult lihtsad väärtused, ühendame need '|' märgiga
        if all(is_scalar(el) for el in obj):
            flat[parent_key] = " | ".join("" if el is None else str(el) for el in obj)
        else:
            # Kui listis on keerulised objektid, jätame selle JSON stringiks, et infot mitte kaotada
            flat[parent_key] = json.dumps(obj, ensure_ascii=False, sort_keys=True)
    else:
        flat[parent_key] = obj
    return flat

def bulletify(items, lang):
    """Teisendab eesmärkide või õpiväljundite listi ilusaks bullet-punktidega tekstiks."""
    out = []
    for it in (items or []):
        txt = (it or {}).get(lang)
        if txt:
            out.append(txt.strip())
    return "\n".join(f"- {t}" for t in out) if out else None

def pick_latest_version(versions: list[dict], latest_uuid: str | None = None) -> dict | None:
    """Valib välja kõige asjakohasema aineversiooni."""
    if not versions:
        return None
    
    # 1. Proovime leida UUID-d, mille andis meile ainenimekirja API
    if latest_uuid:
        for v in versions:
            if v.get("uuid") == latest_uuid:
                return v
                
    # 2. Tagavara: Kontrollime 'isLatest' lippu
    for v in versions:
        if v.get("isLatest") is True:
            return v
            
    # 3. Tagavara: Sorteerime kuupäeva järgi
    def ver_key(v):
        return (v.get("approvalDate") or v.get("validFrom") or "")
    return sorted(versions, key=ver_key, reverse=True)[0]

## Samm 3: Detailse info pärimine ja töötlemine (põhitöö)

See on kogu skripti "süda" ja kõige ajamahukam etapp. Kui 1. sammus saime teada aine ID, siis nüüd peame iga ID kohta tegema lisapäringuid, et saada kätte tegelik sisu (kirjeldused, hindamine jne).

**Mida see koodiplokk täpselt teeb?**

1.  **Loeb sisendi:** Võtab aluseks faili `course_ids.csv`, mille koostasime 1. sammus.
2.  **Topeltpäring iga aine kohta:** Iga aine puhul teeb kood serverile kaks eraldi päringut:
    * **Päring A (konteiner):** Küsib aine üldinfot (kood, nimetus, üldine kirjeldus).
    * **Päring B (versioon):** Küsib konkreetset aineprogrammi (mis semestril aine toimub, kes on õppejõud, täpsed hindamiskriteeriumid).
3.  **Andmete ühildamine:** See on oluline samm. Me võtame üldinfo ja kirjutame selle üle versiooni infoga.
    * *Miks?* Üldinfo võib olla aegunud või liiga üldsõnaline. Konkreetse semestri aineprogramm (versioon) on see, mis tudengit tegelikult huvitab.
4.  **Turvamehhanismid:**
    * **Viisakus (*politeness*):** Kood "magab" iga päringu vahel (`time.sleep`), et mitte koormata ülikooli serverit üle.
    * **Vahesalvestus (*checkpoints*):** Kuna päringud võivad võtta aega (nt 10-15 minutit), salvestab kood tulemused faili `course_details.csv` iga 200 rea järel. Kui internet peaks vahepeal kaduma või arvuti kinni jooksma, on suurem osa tööst juba failis olemas ja ei pea nullist alustama.

**Tulemus:**
Selle sammu lõpuks tekib fail `course_details.csv`, mis on valmis andmeanalüüsiks või projektide sisendiks.

In [6]:
# Seadistused
SLEEP_BETWEEN = 0.1  # Sekundit ootamist päringute vahel (viisakus serveri vastu)
SAVE_EVERY = 200     # Salvesta CSV iga X rea järel

# Laeme nimekirja
try:
    ids_df = pd.read_csv(COURSE_LIST_FILE)
except FileNotFoundError:
    print("Viga: course_ids.csv ei leitud. Palun käivita enne Samm 1.")
    ids_df = pd.DataFrame()

rows = []

print(f"Töötlen {len(ids_df)} ainet...")

for i, r in ids_df.iterrows():
    course_uuid = r["course_uuid"]
    latest_ver_uuid = r.get("latest_version_uuid")

    try:
        # 1. Pärime ja 'lamendame' üldise aineinfo
        raw_course = fetch_course_details(course_uuid)
        flat = flatten_json(raw_course)

        # 2. Lisame loetavad tekstiväljad (hilisemaks NLP töötluseks)
        ov = raw_course.get("overview") or {}
        flat["overview__objectives_text_en"] = bulletify(ov.get("objectives"), "en")
        flat["overview__learning_outcomes_text_en"] = bulletify(ov.get("learning_outcomes"), "en")

        # 3. Pärime versioonid ja valime värskeima
        versions = fetch_versions(course_uuid)
        target_version = pick_latest_version(versions, latest_ver_uuid)
        
        if target_version:
            # Lamendame versiooni andmed prefiksiga 'version__', et neid eristada
            v_flat = flatten_json(target_version, parent_key="version")
            flat.update(v_flat)
            
            # Lisame versiooni loetavad tekstid
            v_ov = target_version.get("overview") or {}
            flat["version__overview__objectives_text_en"] = bulletify(v_ov.get("objectives"), "en")
            flat["version__overview__learning_outcomes_text_en"] = bulletify(v_ov.get("learning_outcomes"), "en")

        # 4. Tagame, et ID-d on selgelt olemas
        flat["course_uuid"] = course_uuid
        flat["latest_version_uuid"] = latest_ver_uuid or (target_version.get("uuid") if target_version else None)

        rows.append(flat)

    except Exception as e:
        print(f"[Viga] Aine {course_uuid}: {e}")
        # Valikuline: traceback.print_exc()

    # Viisakas paus
    time.sleep(SLEEP_BETWEEN)

    # Perioodiline salvestamine (Checkpoint)
    if (i + 1) % SAVE_EVERY == 0:
        pd.DataFrame(rows).to_csv(COURSE_DETAILS_FILE, index=False)
        print(f"[Vahesalvestus] Salvestatud {len(rows)} rida iteratsioonil {i + 1}")

# Lõplik salvestamine
df_final = pd.DataFrame(rows)

# Järjestame veerud nii, et oluline info oleks eespool
priority_cols = [
    "course_uuid", "code", 
    "title__en", "version__title__en", 
    "credits", 
    "overview__description__en", "version__overview__description__en",
    "overview__learning_outcomes_text_en"
]
# Lisame olemasolevad prioriteetsed veerud + ülejäänud
final_cols = [c for c in priority_cols if c in df_final.columns] + [c for c in df_final.columns if c not in priority_cols]
df_final = df_final[final_cols]

df_final.to_csv(COURSE_DETAILS_FILE, index=False)
print(f"\nValmis! Edukalt salvestati {len(df_final)} ainet faili '{COURSE_DETAILS_FILE}'")

Töötlen 3031 ainet...
[Vahesalvestus] Salvestatud 200 rida iteratsioonil 200
[Vahesalvestus] Salvestatud 400 rida iteratsioonil 400
[Vahesalvestus] Salvestatud 600 rida iteratsioonil 600
[Vahesalvestus] Salvestatud 800 rida iteratsioonil 800
[Vahesalvestus] Salvestatud 1000 rida iteratsioonil 1000
[Vahesalvestus] Salvestatud 1200 rida iteratsioonil 1200
[Vahesalvestus] Salvestatud 1400 rida iteratsioonil 1400
[Vahesalvestus] Salvestatud 1600 rida iteratsioonil 1600
[Vahesalvestus] Salvestatud 1800 rida iteratsioonil 1800
[Vahesalvestus] Salvestatud 2000 rida iteratsioonil 2000
[Vahesalvestus] Salvestatud 2200 rida iteratsioonil 2200
[Vahesalvestus] Salvestatud 2400 rida iteratsioonil 2400
[Vahesalvestus] Salvestatud 2600 rida iteratsioonil 2600
[Vahesalvestus] Salvestatud 2800 rida iteratsioonil 2800
[Vahesalvestus] Salvestatud 3000 rida iteratsioonil 3000

Valmis! Edukalt salvestati 3031 ainet faili 'course_details_TEST.csv'


## Samm 4: Valideerimine

Kontrollime tulemuse esimest paari rida, et veenduda andmete korrasolekus.

In [7]:
df = pd.read_csv(COURSE_DETAILS_FILE)
print(f"Andmestiku kuju (read, veerud): {df.shape}")
df.head(3)

Andmestiku kuju (read, veerud): (3031, 223)


,course_uuid,code,title__en,version__title__en,credits,overview__description__en,version__overview__description__en,overview__learning_outcomes_text_en,uuid,state__code,...,overview__description__es,version__title__es,version__overview__description__es,title__de,version__title__de,overview__description__de,title__vro,overview__description__vro,version__title__vro,version__overview__description__vro
0,eefb3a8d-7669-a5dc-9e13-6f95d719ebdf,LOMR.03.021,Practical Course of Microbiology and Virology,Practical Course of Microbiology and Virology,6.0,Gaining practical skills to work with microbes...,Gaining practical skills to work with microbes...,- After the course students are expected to:\n...,eefb3a8d-7669-a5dc-9e13-6f95d719ebdf,confirmed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,76162416-d608-f48f-ec5d-5c40ce9b320d,FLFI.00.016,Doctoral Seminar,Doctoral Seminar,15.0,"Planning, writing and discussing philosophical...","Planning, writing and discussing philosophical...","- On successful completion of the course, the ...",76162416-d608-f48f-ec5d-5c40ce9b320d,confirmed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fdffd50c-3d1b-3517-aaef-c0ea70fa85e0,FLGR.01.138,History of English-Speaking Countries,History of English-Speaking Countries,6.0,The course is an introduction to the main even...,The course is an introduction to the main even...,- By the end of the course the student should ...,fdffd50c-3d1b-3517-aaef-c0ea70fa85e0,confirmed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.version__target__semester__code.value_counts()

version__target__semester__code
autumn    2277
spring     742
Name: count, dtype: int64

In [13]:
df[df.title__en == "Project in Applied Artificial Intelligence"]

,course_uuid,code,title__en,version__title__en,credits,overview__description__en,version__overview__description__en,overview__learning_outcomes_text_en,uuid,state__code,...,overview__description__es,version__title__es,version__overview__description__es,title__de,version__title__de,overview__description__de,title__vro,overview__description__vro,version__title__vro,version__overview__description__vro
